# Lesson 5: Geoparsing in Python

#### Introduction

In this lesson, we will learn how to extract geographical information from text data using a geoparser and visualize the results with sentiment analysis on a map.

---

In [ ]:
from geoparser import Geoparser
from tqdm.notebook import tqdm
import pandas as pd

import warnings

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1 Load necessary files

### 1.1 Load Geoparser

To use Geoparser, instantiate an object of the Geoparser class with optional specifications for the spaCy model, transformer model, and gazetteer. By default, the library uses an accuracy-optimised configuration:

In [ ]:
geo = Geoparser(spacy_model='en_core_web_trf', transformer_model='dguzh/geo-all-distilroberta-v1', gazetteer='geonames')

- `spacy_model`: Controls the spaCy model used for language processing.
- `transformer_model`: Specifies the model for handling transformer-based language parsing.
- `gazetteer`: Defines the database from which geographical names and information are pulled.

#### 1.1.1 Test Geoparser

Geoparser is optimised for parsing large collections of texts at once. To perform parsing, supply a list of strings to the parse method. This method processes the input and returns a list of GeoDoc objects, each containing identified and resolved toponyms:

```python
docs = geo.parse(["Sample text 1", "Sample text 2", "Sample text 3"])
```

Modify your code above to run your own sentences mentioning different types of places or names.


In [ ]:
docs = geo.parse (["The text goes here. New York","There are also locations in Harrisonburg"])

#### 1.1.2 View Geoparser results


We can get the result by cycling through every individual instances of the toponyms in the collectino stored in docs. A toponym is a geographic place name, and the returned data includes comprehensive information like latitude, longitude, and even population for specific types of locations.

In [ ]:
for doc in docs:
    for toponym in doc.toponyms:
        print(toponym)

The toponym variable actually stores a lot of other information as well. The data structure looks like this:

```python
{
'geonameid': 2867714,
'name': 'Munich',
'admin2_geonameid': 2861322,
'admin2_name': 'Upper Bavaria',
'admin1_geonameid': 2951839,
'admin1_name': 'Bavaria',
'country_geonameid': 2921044,
'country_name': 'Germany',
'feature_name': 'seat of a first-order administrative division',
'latitude': 48.13743,
'longitude': 11.57549,
'elevation': None,
'population': 1260391
}
```

Much like a list or a dataframe we can navigate to these individual values by accessing the `.location` value in the toponym, and then going to the individual `key`.

In [ ]:
for doc in docs:
    for toponym in doc.toponyms:
        if toponym.location:
            # If location is resolved
            place_name = toponym
            latitude = toponym.location['latitude']
            longitude = toponym.location['longitude']
            print(f"Place: {place_name}, Latitude: {latitude}, Longitude: {longitude}")
        else:
            # If no location is resolved
            print(f"No location found for toponym: {toponym}")


### 1.2 Load in sentiment data

We can load in the data from last lesson and start there.

In [ ]:
df_virginia_toponyms_sentiment = pd.read_pickle('df_virginia_toponym_sentiment_full.pickle')

As the toponym extraction process is very time intensive. We will run it on a sample, just to show you how it works. The full results will be imported below.

In [ ]:
df_virginia_sample = df_virginia_toponyms_sentiment.head(300).copy()
df_virginia_sample.sample(5)

Since the `geo.parse` function is expecting a list of strings and outputs a complicated piece of data, this logic has been abstracted out into a function to make things a bit simpler. The function goes through and looks at the strings in `cleaned_sentences` and tries to establish a toponym. It also includes a `feature_filter` for only Administrative areas [`A`] (Countries, States, Counties) and Population centers [`P`] because otherwise the process will take very long and also include geographic features like streams, rivers, gulley's etc.

In [ ]:
def geoparse_column(df):
    sentences = df['cleaned_sentences'].tolist()  # Convert column to list
    docs = geo.parse(sentences, feature_filter=['A', 'P'])  # Run geo.parse on the entire list

    # Initialize lists to store the extracted fields
    places, latitudes, longitudes, feature_names = [], [], [], []

    # Iterate through the results and extract toponyms and their locations
    for doc in docs:
        doc_places = []
        doc_latitudes = []
        doc_longitudes = []
        doc_feature_names = []

        for toponym in doc.toponyms:
            if toponym.location:
                doc_places.append(toponym.location.get('name'))
                doc_latitudes.append(toponym.location.get('latitude'))
                doc_longitudes.append(toponym.location.get('longitude'))
                doc_feature_names.append(toponym.location.get('feature_name'))
            else:
                doc_places.append(None)
                doc_latitudes.append(None)
                doc_longitudes.append(None)
                doc_feature_names.append(None)

        # Append the extracted data for the document
        places.append(doc_places)
        latitudes.append(doc_latitudes)
        longitudes.append(doc_longitudes)
        feature_names.append(doc_feature_names)

    # Assign the extracted data to the DataFrame as new columns
    df['place'] = places
    df['latitude'] = latitudes
    df['longitude'] = longitudes
    df['feature_name'] = feature_names

    return df


In [ ]:
geoparse_results_sample = geoparse_column(df_virginia_sample)

In [ ]:
geoparse_results_sample.sample(5, random_state =10)

There are several interesting things of note in the data. First, for some of the sentences the tokenizer did not find a toponym which is indicated by empty lists `[]`. This because this is a more accurate tokenizer and will likely have fewer false positives. We will have to remember to remove these. Likewise, right now the parsing has been set to include Administrative areas like countries and states (i.e. The US and Virginia) and population centers (Richmond, Harrisonburg). We will have to think of how to deal with these down the road.

**We can run the geoparser for all the data and expect to wait at least an hour!**

In [ ]:
# Run the geoparser over the entire 'cleaned_sentences' column
#geoparse_results = geoparse_column(df_virginia_toponyms_sentiment)

In [ ]:
# Display the updated DataFrame with new columns
#geoparse_results.sample(10)

In [ ]:
#geoparse_results.to_pickle('df_virginia_geoparsed_complete.pickle')

---

## 2 Load in geoparsed file

Because the geoparsing process takes so long, we can simply load in the file that has already been processed. Below is how long it took my system:

![](geoparser_completion.png)

In [ ]:
df_virginia_all = pd.read_pickle('df_virginia_geoparsed_complete.pickle')

Note that since the toponym tagger was especially trained for geographic locations it is more accurate. Hence, it also ignores false positives.

In [ ]:
df_virginia_all[['toponyms','place']].sample(7, random_state = 19)

Calculate the number of false positives in the original toponyms.

In [ ]:
empty_percent = (df_virginia_all['place'].str.len() == 0).mean() * 100
print(f"The number of missing values is {empty_percent:.0f}%")

### 2.1 Remove false positives

Here a false positive is defined as any empty value in the more fine-grained value in the toponym parser and a hit in the rough NER extraction.

In [ ]:
df_virginia_cleaned = df_virginia_all[df_virginia_all['place'].str.len() != 0].copy()

In [ ]:
df_virginia_cleaned[['cleaned_sentences','place','latitude','longitude']].sample(5, random_state= 15)

#### 2.1.1 `explode` the dataframe

As previously, we want to get all of the data per row. Some sentences contained multiple locations so we want to get these out.

In [ ]:
df_virginia_long = df_virginia_cleaned.explode(['place', 'latitude', 'longitude', 'feature_name'])

#### 2.1.2 Investigate the data frame

In [ ]:
df_virginia_long[['place', 'latitude', 'longitude', 'feature_name']].sample(5, random_state = 7)

#### 2.1.3 Critical Question

From a cartographic perspective, what is the difference between the types of features recovered?

### 2.2 Remove empty values

We can remove all of the `None` values with `.notna()`. That is, if the value is not `None` return true.

In [ ]:
df_virginia_long = df_virginia_long[df_virginia_long.place.notna()]

When we do this we have to remember to reset the index column.

In [ ]:
df_virginia_long = df_virginia_long.reset_index(drop=True)

Check the result

In [ ]:
df_virginia_long.sample(3, random_state=25)

### 2.3 Creating an Aggregate table with `.groupby()`

We now have a list of every place, latitude, longitude, and type of place it is. This is coupled with the sentiment data. We now need to consolidate this into singular points. We are going to do this by using the `.groupby()` method, which makes the calculation by group. In this case our group is going to be `place`, meaning that for every place it will count the number of times it appears and also the average sentiment score. We group by `place` to consolidate multiple mentions of the same location and calculate the average sentiment to get a general sentiment trend for each location.

In [ ]:
df_geolocations_sentiments = df_virginia_long.groupby('place').agg(
    location_count=('place', 'size'),  # Count occurrences of each location
    latitude=('latitude', 'first'),    # Take the first latitude (you can also use 'mean')
    longitude=('longitude', 'first'),  # Take the first longitude (or 'mean')
    location=('feature_name','first'),
    avg_roberta_pos=('roberta_pos', 'mean'),  # Average of roberta_pos
    avg_roberta_neu=('roberta_neu', 'mean'),  # Average of roberta_neu
    avg_roberta_neg=('roberta_neg', 'mean')   # Average of roberta_neg
).reset_index()



Check the results

In [ ]:
df_geolocations_sentiments.sample(5, random_state=5)

#### 2.3.1 Find top locations

In [ ]:
df_geolocations_sentiments.sort_values(by='location_count', ascending=False).head(5)

What can we tell about the number of frequent locations?

### 2.4 Consolidate the Roberta Score

Since the roberta score is positive, negative, and neutral, we will have to consolidate it into one easier to understand score. We will take the difference between positive and negative, and multiply it by the percentage of neutral. This way if a score is very neutral it will even out the difference between positive and negative.

In [ ]:
# Calculate the compound score and add it as a new column 'roberta_compound'
df_geolocations_sentiments['avg_roberta_compound'] = (
    df_geolocations_sentiments['avg_roberta_pos'] - df_geolocations_sentiments['avg_roberta_neg']
) * (1 - df_geolocations_sentiments['avg_roberta_neu'])


#### 2.4.1 The most negative place

Let's find the most negative place.

In [ ]:
# Sort the DataFrame by 'roberta_compound' in ascending order and display the top 10 negative scores
top_10_negative = df_geolocations_sentiments.sort_values(by='avg_roberta_compound').head(10)
top_10_negative

### 2.4.2 Critical Question

We can already tell there might be some challenges here. The values with the strongest scores tend to be low in count. 

- Why might this be?

Also, these are different types of features. 

- How might this distort results?

In [ ]:
#df_geolocations_sentiments.to_pickle('df_geolocations_sentiments.pickle')

##### 3 (Optional) Load in completed file

In [ ]:
df_geolocations_sentiments = pd.read_pickle('df_geolocations_sentiments.pickle')

---

## 3 Mapping Sentiment

In the early days of digital mapping, getting map on the web was truly a lot of labor. You would have to map the data in a local piece of software, then export it to a format available for web viewing, using JavaScript to make it interactive, and then create your own custom webpage. Now we can do it with only a few lines of code using `plotly` which is a powerful visualization library that also has access to `mapbox` a very powerful piece of mapping software.

### 3.1 Basic Plotly Map

Below is the basic stub from the instructions to create a [map](https://plotly.com/python/tile-scatter-maps/). 

In [ ]:
import plotly.express as px

# Create the map using plotly.express 
fig = px.scatter_mapbox(
    df_geolocations_sentiments,  #put your dataframe here
    lat="latitude",               # Latitude column
    lon="longitude",              # Longitude column
    size="location_count",        # Bubble size based on location count
    color="avg_roberta_compound",      # Color based on sentiment score
    color_continuous_scale=px.colors.cyclical.IceFire,  # Use IceFire scale (blue to red)
    size_max=15,                  # Maximum size of the bubbles
    center={"lat": 0.0, "lon": 0.0},
    zoom=1                        # Adjust zoom level for better visibility
)

# Update the layout to use the default map style (which doesn't need a token)
fig.update_layout(
    mapbox_style="open-street-map",  # No token needed for this style
    margin={"r":0,"t":0,"l":0,"b":0}  # Remove margins for a cleaner view
)

fig.show()

### 3.1 Critical Questions

- What are some issue with this map?
- How can we make it better?

#### 3.2.1 Adjust the position and the zoom

Look at the comments next to each variable in the plotly function, what does each thing do? 
- If I wanted to get a closer zoom how would I fix it?
- If I want to set a different center what should I choose?
- How would I change the size of the bubbles?
- How can I get a different mapbox style?

Take some time to mess around with your map.


[Read the full documentation](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html)

In [ ]:
# Create the map using plotly.express 
fig = px.scatter_mapbox(
    df_geolocations_sentiments,  #put your dataframe here
    lat="latitude",               # Latitude column
    lon="longitude",              # Longitude column
    size="location_count",        # Bubble size based on location count
    color="avg_roberta_compound",      # Color based on sentiment score
    color_continuous_scale=px.colors.cyclical.IceFire,  # Use IceFire scale (blue to red)
    size_max=50,                  # Maximum size of the bubbles
    center={"lat": 0.0, "lon": 0.0},
    zoom=1                        # Adjust zoom level for better visibility
)

# Update the layout to use the default map style (which doesn't need a token)
fig.update_layout(
    mapbox_style="open-street-map",  # No token needed for this style
    margin={"r":0,"t":0,"l":0,"b":0}  # Remove margins for a cleaner view
)

fig.show()

### 3.3 Bubble Size

One major issue is simply the size of the bubbles. The data is very spread out. The lowest number is 1 and the highest number is 11000. We can make life a little easier by simply removing some of the lower numbers. We can do this randomly by simply removing every below a certain count or we can be a bit more thoughtful and only consider a certain percentage of values. We can actually get a nice summary of a column using the `.describe()` function.



In [ ]:
df_geolocations_sentiments.location_count.describe()

It actually looks like a significant number of locations appear very few times. We will have to keep this in mind going forward. For now, let's set our count threshold to 100.

### 3.4 Critical Question

Create a new dataframe that sets the minimum location count to 100.

In [ ]:
df_geolocations_sentiments_small = df_geolocations_sentiments[df_geolocations_sentiments.location_count>99]

In [ ]:
# Create the map using plotly.express 
fig = px.scatter_mapbox(
    df_geolocations_sentiments_small,  #put your dataframe here
    lat="latitude",               # Latitude column
    lon="longitude",              # Longitude column
    size="location_count",        # Bubble size based on location count
    color="avg_roberta_compound",      # Color based on sentiment score
    color_continuous_scale=px.colors.cyclical.IceFire,  # Use IceFire scale (blue to red)
    size_max=20,                  # Maximum size of the bubbles
    center={"lat": 37.5246322, "lon": -77.5758331},
    zoom=4                        # Adjust zoom level for better visibility
)

# Update the layout to use the default map style (which doesn't need a token)
fig.update_layout(
    mapbox_style="carto-darkmatter",  # No token needed for this style
    margin={"r":0,"t":0,"l":0,"b":0}  # Remove margins for a cleaner view
)



fig.show()

### 3.5 Critical Question

- How did reducing the number of values improve the data visualization?
- Where does it run into issues?

### 3.6 Bucketing Values

The above map still runs into a sizing issue. This is because even though the values are a bit closer together there is still a difference of 11,000 between the lowest value and the highest value. This is a common problem when representing values on a map. The easiest way to fix this is to "bucket" the values from "smallest" to "small" "medium", "large", and "largest". The problem is deciding how to do this in the most optimal way. 

   - **Equal Interval**: This method divides the entire range of values into equal-sized buckets. It’s simple to implement and works well when data is evenly distributed. However, it may not be as effective when the data is heavily skewed, as it can lead to many data points clustering within certain buckets.
   - **Quantile (Percentiles)**: Quantile bucketing divides data so that each bucket contains an equal number of data points. This method is useful for data with uneven distributions, as it ensures that each category has a similar representation.
   - **Natural Breaks (Jenks)**: The Jenks method automatically identifies clusters and gaps within the data to create buckets based on natural groupings. This technique is particularly beneficial for data with distinct groupings, as it helps to highlight these patterns and produce visually distinct buckets that better reflect the distribution of values.


We are going to go with natural breaks using `mapclassify`.

In [ ]:
import mapclassify as mc #you may get an error. If so install mapclassify with pip install mapclassify

jenks_breaks = mc.NaturalBreaks(y=df_geolocations_sentiments_small['location_count'], k=5)
jenks_breaks

#### 3.6.1 Critical Question

Now that we have a better view of the data, what would it have looked like with equal interval?

In [ ]:
# Create a new column 'location_count_bucket' for the classified values
df_geolocations_sentiments_small.loc[:,'location_count_bucket'] = jenks_breaks.find_bin(df_geolocations_sentiments_small['location_count'])+1

#### 3.6.2 Optional Retrieve Backup

get copy of `df_geolocations_sentiments_small.pickle` if the above does not work.

In [ ]:
#df_geolocations_sentiments_small = pd.read_pickle("df_geolocations_sentiments_small.pickle")

#### 3.6.3 Explore Buckets

In [ ]:
df_geolocations_sentiments_small.sample(5, random_state=4)

What do we notice about the buckets?

In [ ]:
# Create the map using plotly.express 
fig = px.scatter_mapbox(
    df_geolocations_sentiments_small,  #put your dataframe here
    lat="latitude",               # Latitude column
    lon="longitude",              # Longitude column
    size="location_count_bucket",        # Bubble size based on location count
    color="avg_roberta_compound",      # Color based on sentiment score
    color_continuous_scale=px.colors.cyclical.IceFire,  # Use IceFire scale (blue to red)
    size_max=20,                  # Maximum size of the bubbles
    center={"lat": 37.5246322, "lon": -77.5758331},
    zoom=4                        # Adjust zoom level for better visibility
)

# Update the layout to use the default map style (which doesn't need a token)
fig.update_layout(
    mapbox_style="carto-darkmatter",  # No token needed for this style
    margin={"r":0,"t":0,"l":0,"b":0}  # Remove margins for a cleaner view
)



fig.show()


### 3.7 Improving labels and colors

Now the markers need some labels and the colors need to be reversed and need a new midpoint.

We can set the midpoint for the color scale with the argument:

```python
color_continuous_scale=0,
```

We can reverse the `IceFire` color scale using reverse list slicing 

```python
[::-1]
```

Or you can pick a different one [here](https://plotly.com/python/builtin-colorscales/)




In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(
    df_geolocations_sentiments_small,  # DataFrame
    lat="latitude",                   # Latitude column
    lon="longitude",                  # Longitude column
    size="location_count_bucket",     # Bubble size based on location count bucket
    color="avg_roberta_compound",     # Color based on sentiment score
    color_continuous_scale=px.colors.cyclical.IceFire[::-1],  # IceFire color scale
    color_continuous_midpoint=0,      # Set 0 as the center point
    size_max=20,                      # Maximum size of the bubbles
    center={"lat": 37.5246322, "lon": -77.5758331},
    zoom=4,                           # Zoom level
    hover_name="place",               # Show place name
    hover_data={
            "avg_roberta_compound": ':.2f',   # Rounded sentiment score to 2 decimals
        "longitude": False,
        "latitude": False,
        "location_count_bucket": False
    }
)

# Update the layout to use the default map style
fig.update_layout(
    mapbox_style="carto-darkmatter",  # No token needed
    margin={"r": 0, "t": 0, "l": 0, "b": 0}  # Remove margins for cleaner view
)

# Show the plot
fig.show()


How else can this map be improved?

In [ ]:
df_geolocations_sentiments_small.to_pickle('df_geolocations_sentiments_small.pickle')